<a href="https://colab.research.google.com/github/giriragav/UChicago---QCSD/blob/main/Module%203/lab6/lab6_schedulling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>